In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import scipy.integrate 
from ipywidgets import Video

In [14]:
%%capture

def f(x,y):
  if (x-L/2)**2+(y-H/2)**2 < 1:
    return -((x-L/2)**2+(y-H/2)**2-1)
  else:
    return 0

def fi(x,y):
  """coefficients A_m,n"""
  return f(x,y)*np.sin(n*np.pi*x/L)*np.sin(m*np.pi*y/H)*4/(L*H)

def g(x,y):
  return 0

def gi(x,y):
  """coefficients B_m,n"""
  return g(x,y)*np.sin(n*np.pi*x/L)*np.sin(m*np.pi*y/H)*4/(np.pi*c*np.sqrt(m**2*L**2+n**2*H**2))

L=5
H=5
j=5
k=5
c=1
tot_frames=750

A_coeff=[]
B_coeff=[]

for n in range(1,k+1):
  for m in range(1,j+1):
    A_coeff.append(scipy.integrate.dblquad(fi,0,H,0,L)[0])
    B_coeff.append(scipy.integrate.dblquad(gi,0,H,0,L)[0])

A_coeff=np.array(A_coeff)
A_coeff=A_coeff.reshape((k,j,1,1))
B_coeff=np.array(B_coeff)
B_coeff=B_coeff.reshape((k,j,1,1))

x=np.linspace(0,L,250)
y=np.linspace(0,H,250)
X,Y=np.meshgrid(x,y)

sines=np.array([[np.sin(n*np.pi*X/L)*np.sin(m*np.pi*Y/H) for m in range(1,j+1)] 
          for n in range(1,k+1)])
NTDA=A_coeff*sines
NTDB=B_coeff*sines
NTDA=NTDA.reshape((k,j,len(x),len(y)))
NTDB=NTDB.reshape((k,j,len(x),len(y)))

sq=np.array([[np.sqrt(m**2/H**2+n**2/L**2)*np.pi*c/25 for m in range(1,j+1)]
            for n in range(1,k+1)])

z=np.array([np.sum([np.array(NTDA[n-1,m-1]*np.cos(sq[n-1,m-1]*t)+
                    NTDB[n-1,m-1]*np.sin(sq[n-1,m-1]*t)) 
                    for n in range(1,k+1) for m in range(1,j+1)],axis=0) 
  for t in range(tot_frames)])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
if L > H:
  lim=L
else:
  lim=H

def init():
  ax.set_xlim(0,lim)
  ax.set_ylim(0,lim)
  ax.set_zlim(1.2*np.min(z),1.2*np.max(z))
  ax.set_xlabel("x",fontsize=18)
  ax.set_ylabel("y",fontsize=18)
  ax.set_zlabel("u(x,y,t)",fontsize=14)

def update_plot(frame,z,plot):
  plot[0].remove()
  plot[0]=ax.plot_surface(X,Y,z[frame],cmap="magma",linewidth=0)

plot=[ax.plot_surface(X,Y,z[0],cmap="magma",linewidth=0)]
ani1 = ani.FuncAnimation(fig, update_plot, frames=range(tot_frames),
                         init_func=init, repeat=False, fargs=(z, plot), interval=40)

ani1.save('2D_parabolic_wave.mp4')

In [16]:
Video.from_file("2D_parabolic_wave.mp4",loop=False)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0b\xe4@mdat\x00\x…